# ccdproc-02:   Trimming

## Overview
1. Build a list of all image files
2. How to trim images 
3. Trim all the images to the selected region of interest

<pre>
Máster en Astrofísica UCM  -- Técnicas Experimentales en Astrofísica  
Jaime Zamorano, Nicolás Cardiel and Sergio Pascual

This notebook has reproduced parts of the the astropy ccdproc docs: 
https://ccdproc.readthedocs.io/en/latest/reduction_toolbox.html#subtract-overscan-and-trim-images

Version 1.0 2021/01/14  
Version 2.0 2021/01/18  
</pre>

Note that the ``astropy`` package should be installed. In this sense, have a look to the
astropy installation description: https://docs.astropy.org/en/stable/install.html.

In [ ]:
from astropy.io import fits
from pylab import *
import matplotlib
%matplotlib inline
import matplotlib.pyplot as plt
import matplotlib.mlab as ml
import numpy as np
from mpl_toolkits.axes_grid1 import make_axes_locatable

In [ ]:
plt.style.use('./tea.mplstyle')   # Some parameters for nicer graphs

### Working with images in a directory
We will use the files of the first night of observations at NOT (Nordic Optical Telescope) 2008 that can be downloaded from   ftp://astrax.fis.ucm.es/pub/users/jaz/NOT_2008_04_12-14/N1/
or 
http://guaix.fis.ucm.es/~jaz/master_TEA/observaciones_NOT_2008/N1/

``directory`` should point to the directory with the downloaded images

In [ ]:
directory='NOT_2008_N1/'

Let\'s create a list containing all the FITS files in that directory 

In [ ]:
import os
from glob import glob
# os.path.join is a platform-independent way to join two directories
globpath = os.path.join(directory, '*.fits')
print(globpath)
# glob searches through directories similar to the Unix shell
filelist = sorted(glob(globpath))
print(filelist[100:120])    # printing only from 10 to 20

Let\'s open one of the files to explore the HEADER keywords

In [ ]:
HDUList_object = fits.open(filelist[101])
HDUList_object.info()

In [ ]:
# Extract primary header
primary_header = HDUList_object[0].header
# Index header object with keyword name and print value
print('filename:',primary_header['FILENAME'],'    Object:',primary_header['OBJECT'])
# Read the data to a numpy array
image = fits.getdata(filelist[101])
print(type(image))
print('dimension:',image.shape)
naxis2, naxis1 = image.shape
print('naxis1:',image.shape[1])
print('naxis2:',image.shape[0])

Let\'s display this file

In [ ]:
fig, ax = plt.subplots(ncols=1, nrows=1, figsize=(8, 8))
img = ax.imshow(image, cmap='gray', origin='low',vmin=4000,vmax=6500)
ax.set_xlabel('X axis')
ax.set_ylabel('Y axis')
divider = make_axes_locatable(ax)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Number of counts')
ax.grid()

Selecting a region of the image can be done using ``trimming`` or simply using the python command.

### Trimming using python 

In [ ]:
new_image = image[0:1000,0:1250]

In [ ]:
fig, axarr = plt.subplots(ncols=2, nrows=1, figsize=(12, 8))
ax0 = axarr[0]
img = ax0.imshow(image, cmap='gray', origin='low',vmin=4000,vmax=6500)
ax0.set_xlabel('X axis')
ax0.set_ylabel('Y axis')
ax0.grid()
ax1 = axarr[1]
img = ax1.imshow(new_image, cmap='gray', origin='low',vmin=4000,vmax=6500)
ax1.set_xlabel('X axis')
ax1.set_ylabel('Y axis')
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Number of counts')
ax1.grid()

In [ ]:
### Trimming using python the region of interest TRIMSEC= [20:-20,60:2100]
new_image = image[20:-20,60:2100]
print(image.shape,new_image.shape)

In [ ]:
fig, axarr = plt.subplots(ncols=2, nrows=1, figsize=(12, 8))
ax0 = axarr[0]
img = ax0.imshow(image, cmap='gray', origin='low',vmin=4000,vmax=6500)
ax0.set_xlabel('X axis')
ax0.set_ylabel('Y axis')
ax0.grid()
ax1 = axarr[1]
img = ax1.imshow(new_image, cmap='gray', origin='low',vmin=4000,vmax=6500)
ax1.set_xlabel('X axis')
ax1.set_ylabel('Y axis')
divider = make_axes_locatable(ax1)
cax = divider.append_axes("right", size="5%", pad=0.05)
fig.colorbar(img, cax=cax, label='Number of counts')
ax1.grid()

### Trimming with ccdproc trim_image
We need to convert the numpy array to [CCDData](https://docs.astropy.org/en/stable/api/astropy.nddata.CCDData.html) class

In [ ]:
from astropy import units as u
from astropy.nddata import CCDData
import ccdproc

In [ ]:
data_image = CCDData(image,unit=u.adu)

In [ ]:
# FITS-style
t_image_1 = ccdproc.trim_image(data_image,fits_section='[60:2099, 21:2032]')
# python-style:
t_image_2 = ccdproc.trim_image(data_image[20:-20,60:2100])
print(t_image_1.shape,t_image_2.shape)

In [ ]:
fig, axarr = plt.subplots(ncols=2, nrows=1, figsize=(12, 8))
ax0 = axarr[0]
img = ax0.imshow(t_image_1, cmap='gray', origin='low',vmin=4000,vmax=6500)
ax0.set_xlabel('X axis')
ax0.set_ylabel('Y axis')
ax0.grid()
ax1 = axarr[1]
img = ax1.imshow(t_image_2, cmap='gray', origin='low',vmin=4000,vmax=6500)
ax1.set_xlabel('X axis')
ax1.set_ylabel('Y axis')
ax1.grid()

### Writing the resulting image as FITS file

We can write the resulting CCDdata file to a FITS file.  
The trim_image command maintain the information of the header (metadata) and also add some information. We will add more information.

In [ ]:
# Replace FILENAME keyword and add information
t_image_1.header['FILENAME']  = 't_ALrd120102.fits' 
t_image_1.header['TRIMIM']  = 'fits_section=[60:2099, 21:2032]' 
t_image_1.header['HISTORY'] = str(datetime.datetime.now())[0:18]+' astropy ccdproc trim_image'

In [ ]:
t_image_1.write('dummy.fits',overwrite='yes')

## Trimming all the images of the first night

We are using this method to trim all the images of the first night of the NOT2008 campaing.  
We will read all the files to create CCDData objects. We created earlier in this notebook the list of files as filelist.

In [ ]:
for i in range(len(filelist)):
    image = CCDData.read(filelist[i], unit="adu")
    #print(i,image.header)
    t_image = ccdproc.trim_image(image[20:-20,60:2100])
    #print(t_image.header)
    name_of_file = 't_'+ str(image.header['FILENAME'])
    t_image.header['FILENAME']  = name_of_file
    t_image.header['HISTORY'] = str(datetime.datetime.now())[0:18]+' astropy ccdproc trim_image'
    t_image.header['HISTORY']  = 'trimming fits_section=[60:2099, 21:2032]' 
    print('writting '+name_of_file+ ' in '+directory)
    t_image.write(directory+name_of_file,overwrite='yes')